In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/housing/target.csv
/kaggle/input/housing/train.csv
/kaggle/input/housing/test.csv
/kaggle/input/housing/AmesHousing.csv


In [2]:
df = pd.read_csv("/kaggle/input/housing/AmesHousing.csv")
df.info()
datalen = len(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 82 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Order            2930 non-null   int64  
 1   PID              2930 non-null   int64  
 2   MS SubClass      2930 non-null   int64  
 3   MS Zoning        2930 non-null   object 
 4   Lot Frontage     2440 non-null   float64
 5   Lot Area         2930 non-null   int64  
 6   Street           2930 non-null   object 
 7   Alley            198 non-null    object 
 8   Lot Shape        2930 non-null   object 
 9   Land Contour     2930 non-null   object 
 10  Utilities        2930 non-null   object 
 11  Lot Config       2930 non-null   object 
 12  Land Slope       2930 non-null   object 
 13  Neighborhood     2930 non-null   object 
 14  Condition 1      2930 non-null   object 
 15  Condition 2      2930 non-null   object 
 16  Bldg Type        2930 non-null   object 
 17  House Style   

In [3]:
# To check if any column data has more than half of NaN value ready to dispose. 
def drop_na(df):
    dropna_list = []
    for i in df:
        d = dict(df[i].value_counts(dropna=False))
        if (np.nan in d):
            if (d[np.nan] > datalen):
                dropna_list.append(i)
    return dropna_list
        

In [4]:
drop_na(df)

[]

In [5]:
# This is the list that manually drop columns that is unnecassary

drop_list = ["Foundation", "Mas Vnr Type", "MS Zoning", "Electrical", "Order", "PID", "MS SubClass", "Street", "Alley", "Land Contour", "Utilities",
             "Lot Config", "Land Slope", "Neighborhood", "Condition 1", "Condition 2", "Bldg Type",
             "Roof Style", "Roof Matl", "Exterior 1st", "Exterior 2nd", "Bsmt Qual", "Bsmt Cond",
             "Bsmt Exposure", "Heating", "Central Air", "Low Qual Fin SF", "Functional", "Fireplace Qu",
             "Garage Type", "Garage Yr Blt","Garage Qual", "Garage Cond", "Paved Drive", "3Ssn Porch", "Pool Area", "Pool QC",
             "Pool QC", "Fence", "Misc Feature", "Misc Val", "Mo Sold", "Yr Sold", "Sale Type", 
             "Sale Condition", "Lot Shape"]
df = df.drop(drop_list, axis=1)

In [6]:
df.iloc[0]

Lot Frontage       141.0
Lot Area           31770
House Style       1Story
Overall Qual           6
Overall Cond           5
Year Built          1960
Year Remod/Add      1960
Mas Vnr Area       112.0
Exter Qual            TA
Exter Cond            TA
BsmtFin Type 1       BLQ
BsmtFin SF 1       639.0
BsmtFin Type 2       Unf
BsmtFin SF 2         0.0
Bsmt Unf SF        441.0
Total Bsmt SF     1080.0
Heating QC            Fa
1st Flr SF          1656
2nd Flr SF             0
Gr Liv Area         1656
Bsmt Full Bath       1.0
Bsmt Half Bath       0.0
Full Bath              1
Half Bath              0
Bedroom AbvGr          3
Kitchen AbvGr          1
Kitchen Qual          TA
TotRms AbvGrd          7
Fireplaces             2
Garage Finish        Fin
Garage Cars          2.0
Garage Area        528.0
Wood Deck SF         210
Open Porch SF         62
Enclosed Porch         0
Screen Porch           0
SalePrice         215000
Name: 0, dtype: object

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 37 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Lot Frontage    2440 non-null   float64
 1   Lot Area        2930 non-null   int64  
 2   House Style     2930 non-null   object 
 3   Overall Qual    2930 non-null   int64  
 4   Overall Cond    2930 non-null   int64  
 5   Year Built      2930 non-null   int64  
 6   Year Remod/Add  2930 non-null   int64  
 7   Mas Vnr Area    2907 non-null   float64
 8   Exter Qual      2930 non-null   object 
 9   Exter Cond      2930 non-null   object 
 10  BsmtFin Type 1  2850 non-null   object 
 11  BsmtFin SF 1    2929 non-null   float64
 12  BsmtFin Type 2  2849 non-null   object 
 13  BsmtFin SF 2    2929 non-null   float64
 14  Bsmt Unf SF     2929 non-null   float64
 15  Total Bsmt SF   2929 non-null   float64
 16  Heating QC      2930 non-null   object 
 17  1st Flr SF      2930 non-null   i

In [8]:
replacement = {    
#     np.nan     : -1,
    "NA"       : 0,
    "None"     : 0,
    "SFoyer"   : 0,
    "Unf"      : 0.25,
    "SLvl"     : 0.5,
    "RFn"      : 0.75,
    "LwQ"      : 1,
    "1Story"   : 1,
    "Fin"      : 1,
    "Stone"    : 1,
    "Po"       : 1,
    "Fa"       : 1,
    "BrkTil"   : 1,
    "BrkCmn"   : 1,
    "1.5Unf"   : 1.25,
    "1.5Fin"   : 1.5,
    "Stone"    : 10,
    "BLQ"      : 2,
    "2Story"   : 2,
    "Fa"       : 2,
    "2.5Unf"   : 2.25,
    "2.5Fin"   : 2.5,
    "TA"       : 3,
    "ALQ"      : 3,
    "GLQ"      : 4,
    "Gd"       : 4,
    "Ex"       : 5,
    "Rec"      : 99
}

    4  GLQ	Good Living Quarters
    3  ALQ	Average Living Quarters
    2  BLQ	Below Average Living Quarters
    99  Rec	Average Rec Room
    1    LwQ	Low Quality
    0.25 Unf	Unfinshed
    0.0  NA 	No Basement

In [9]:
df = df.replace(replacement)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 37 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Lot Frontage    2440 non-null   float64
 1   Lot Area        2930 non-null   int64  
 2   House Style     2930 non-null   float64
 3   Overall Qual    2930 non-null   int64  
 4   Overall Cond    2930 non-null   int64  
 5   Year Built      2930 non-null   int64  
 6   Year Remod/Add  2930 non-null   int64  
 7   Mas Vnr Area    2907 non-null   float64
 8   Exter Qual      2930 non-null   int64  
 9   Exter Cond      2930 non-null   int64  
 10  BsmtFin Type 1  2850 non-null   float64
 11  BsmtFin SF 1    2929 non-null   float64
 12  BsmtFin Type 2  2849 non-null   float64
 13  BsmtFin SF 2    2929 non-null   float64
 14  Bsmt Unf SF     2929 non-null   float64
 15  Total Bsmt SF   2929 non-null   float64
 16  Heating QC      2930 non-null   int64  
 17  1st Flr SF      2930 non-null   i

In [13]:
df.to_csv('ames_dataset.csv', index=False)

In [10]:
# sns.pairplot(df)

In [11]:
# df.to_csv("ames_housing.csv")